In [14]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

import os

In [15]:
from pathlib import Path

# Configurable parts
SYMBOL = "USDJPY"
SAMPLE_TYPE = "time"
MINUTES = 1
DOLLAR_THRESHOLD = "115009542m"

START_DATE = "20210101"
END_DATE = "20241231"

# Build base name
if SAMPLE_TYPE == "dollar":
    RESAMPLED_NAME = f"{SYMBOL}-{DOLLAR_THRESHOLD}-dollar-{START_DATE}-{END_DATE}"
else:
    RESAMPLED_NAME = f"{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"

# Base directories
BASE_DIR = Path("../data")
RESAMPLED_DIR = BASE_DIR / "resampled"
PROCESSED_DIR = BASE_DIR / "processed"
EVENTS_DIR = BASE_DIR / "events"

# Final paths
RESAMPLED_FILE_PATH = RESAMPLED_DIR / f"{RESAMPLED_NAME}.pkl"
PROCESSED_FILE_PATH = PROCESSED_DIR / f"{RESAMPLED_NAME}-processed.pkl"

In [16]:
%%time
df = pd.read_pickle(PROCESSED_FILE_PATH)

CPU times: user 2.01 ms, sys: 151 ms, total: 153 ms
Wall time: 158 ms


In [17]:
df.shape

(1487207, 52)

In [18]:
df.head()

,open,high,low,close,volume,spread,close_delta,close_return,close_log_return,ret_mean_5,...,dom,month,hour_sin,hour_cos,dow_sin,dow_cos,dom_sin,dom_cos,month_sin,month_cos
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-03 23:44:00,103.2920,103.3005,103.2875,103.2890,1.147400e+11,0.003725,-0.0030,-0.000029,-0.000029,-0.000019,...,3,1,-0.258819,0.965926,-0.781831,0.62349,0.571268,0.820763,0.5,0.866025
2021-01-03 23:45:00,103.2860,103.3090,103.2835,103.3040,2.637000e+11,0.004039,0.0150,0.000145,0.000145,0.000022,...,3,1,-0.258819,0.965926,-0.781831,0.62349,0.571268,0.820763,0.5,0.866025
2021-01-03 23:46:00,103.3045,103.3180,103.2980,103.2980,1.622600e+11,0.003704,-0.0060,-0.000058,-0.000058,0.000010,...,3,1,-0.258819,0.965926,-0.781831,0.62349,0.571268,0.820763,0.5,0.866025
2021-01-03 23:47:00,103.3010,103.3015,103.2805,103.2895,1.991100e+11,0.003133,-0.0085,-0.000082,-0.000082,-0.000005,...,3,1,-0.258819,0.965926,-0.781831,0.62349,0.571268,0.820763,0.5,0.866025
2021-01-03 23:48:00,103.2855,103.2855,103.2690,103.2825,1.275100e+11,0.002381,-0.0070,-0.000068,-0.000068,-0.000018,...,3,1,-0.258819,0.965926,-0.781831,0.62349,0.571268,0.820763,0.5,0.866025


In [19]:
df.columns

Index(['open', 'high', 'low', 'close', 'volume', 'spread', 'close_delta',
       'close_return', 'close_log_return', 'ret_mean_5', 'ret_mean_10',
       'log_volume', 'ema5', 'ema5_slope', 'ema20', 'ema20_slope', 'ema100',
       'ema100_slope', 'atr14', 'atr20', 'atr50', 'vol_adj_return',
       'close_to_atr', 'adx14', 'bb_upper', 'bb_lower', 'bb_mavg', 'bb_width',
       'bb_position', 'donchian_upper', 'donchian_lower', 'donchian_mid',
       'donchian_width', 'stoch_k', 'stoch_d', 'rsi14', 'macd', 'macd_signal',
       'macd_diff', 'unix_time', 'hour', 'dow', 'dom', 'month', 'hour_sin',
       'hour_cos', 'dow_sin', 'dow_cos', 'dom_sin', 'dom_cos', 'month_sin',
       'month_cos'],
      dtype='object')

## Calculate Daily Volatility

In [20]:
vol = df['close_log_return'].rolling(window=60).std() # 計算 60 分鐘的 標準差
vol.mean()

np.float64(0.0001261215076018857)

## Get Events Start Time (t0)

In [21]:
import numpy as np
import pandas as pd
from tqdm import tqdm

def get_tevents_optimized(data: pd.Series, threshold: float) -> pd.DatetimeIndex:
    # get tEvents using CUSUM filter
    values = data.values
    timestamps = data.index

    s_pos = np.zeros_like(values)
    s_neg = np.zeros_like(values)

    t_events_mask = np.zeros_like(values, dtype=bool)

    cum_pos, cum_neg = 0.0, 0.0

    for i in tqdm(range(len(values))):
        cum_pos = max(0.0, cum_pos + values[i])
        cum_neg = min(0.0, cum_neg + values[i])
        s_pos[i] = cum_pos
        s_neg[i] = cum_neg

        if cum_pos > threshold:
            t_events_mask[i] = True
            cum_pos = 0.0
        if cum_neg < -threshold:
            t_events_mask[i] = True
            cum_neg = 0.0

    return timestamps[t_events_mask]


In [22]:
EVENT_NAME = 'CUSUM'
THRESHOLD = vol.mean()*2
EVENT_PATH = EVENTS_DIR / f"{RESAMPLED_NAME}_{EVENT_NAME}_{THRESHOLD:.2e}.pkl"

In [23]:
%%time

t_events = get_tevents_optimized(df['close_log_return'].iloc[1:], threshold=THRESHOLD)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1487206/1487206 [00:01<00:00, 1208186.44it/s]

CPU times: user 1.17 s, sys: 45.1 ms, total: 1.21 s
Wall time: 1.24 s


In [24]:
t_events

DatetimeIndex(['2021-01-03 23:49:00', '2021-01-03 23:52:00',
               '2021-01-04 00:01:00', '2021-01-04 00:05:00',
               '2021-01-04 00:10:00', '2021-01-04 00:17:00',
               '2021-01-04 00:27:00', '2021-01-04 00:30:00',
               '2021-01-04 00:37:00', '2021-01-04 00:40:00',
               ...
               '2024-12-30 22:29:00', '2024-12-30 23:00:00',
               '2024-12-30 23:06:00', '2024-12-30 23:10:00',
               '2024-12-30 23:12:00', '2024-12-30 23:17:00',
               '2024-12-30 23:18:00', '2024-12-30 23:35:00',
               '2024-12-30 23:40:00', '2024-12-30 23:47:00'],
              dtype='datetime64[ns]', name='timestamp', length=248989, freq=None)

In [25]:
EVENT_PATH

PosixPath('../data/events/USDJPY-1m-20210101-20241231_CUSUM_2.52e-04.pkl')

In [26]:
t_events.to_series().to_pickle(EVENT_PATH)